## Langchain + OpenAI LLM tracing using Maxim + Langchain

In [ ]:

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

In [2]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Here we will initialize MaximLangchainTracer

In [ ]:
from maxim import Maxim

logger = Maxim().logger()

Setting up langchain call

In [ ]:
import json
import uuid
from langchain_openai import ChatOpenAI
from maxim.logger.langchain import MaximLangchainTracer
from langchain_core.tools import tool

langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o"
llm = ChatOpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY)


# Define a simple tool
@tool
def search_tool(query: str) -> str:
     """
     Search the web for the latest news.

    Args:
        query: first int        
    """
     return f"Search results for: {query}"


user_input = "latest football news"
system_message = "You are a helpful assistant."

llm_with_tools = llm.bind_tools([search_tool])
trace_id = str(uuid.uuid4())
trace = logger.trace({"id": trace_id, "name": "langchain_trace"})
trace.end()
# Make the API call to Claude using LangChain
messages = [("system", system_message), ("human", user_input)]
response = llm_with_tools.invoke(messages,config={"callbacks":[langchain_tracer],"metadata":{"maxim":{"trace_id":trace.id}}})
print(response)
response_text = response.content
search_toll_call = trace.tool_call({"id": str(uuid.uuid4()),"name": "search_tool","args":json.dumps({"query":user_input})})
# execute the tool
final_tool_call_response = "this is tool result"
search_toll_call.result(final_tool_call_response)
print(response_text)
trace.set_output(final_tool_call_response)


# service 2
trace = logger.trace({"id": trace_id, "name": "langchain_trace"})
span_id=str(uuid.uuid4())
span = trace.span({"id": span_id,"name": "langchain_span"})

response = llm_with_tools.invoke(messages,config={"callbacks":[langchain_tracer],"metadata":{"maxim":{"span_id":span_id}}})
span.event(str(uuid.uuid4()),"langchain_event")
span.end()


In [ ]:
langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o"
llm = ChatOpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY, streaming=True)

user_input = "Describe big bang theory"
system_message = "You are a helpful assistant."
# Make the API call using LangChain with streaming
messages = [("system", system_message), ("human", user_input)]

# Stream the response
response_text = ""
for chunk in llm.stream(messages, config={"callbacks":[langchain_tracer]}):
    pass

print("\n\nFull response:", response_text)